# From Prev.
#Rearranged code
#Used ไทยรัฐฉบับพิมพ์ and ไทยรัฐออนไลน์ 460 data examples for train+test set
#Match Similar words manually (due to limited size of categories [hopefully :P])
#text = re.sub("[^ก-๙]", '', text) 73.04% with TFIDF
#text = re.sub("[^ก-ไ๐-๙]", '', text) ~72% with TFIDF
#used news_content instead of news_label, pretty fast

In [1]:
import pandas as pd
from urllib.parse import unquote

df = pd.read_json('data-1.json',encoding='utf-8')
#df.drop(['date','body','author','url'],axis=1,inplace=True) #for test_2.json
#df.drop(['id','news_content','date','url'],axis=1,inplace=True)

In [2]:
CAT= {
    #category: change to this
    'ข่าวบันเทิง':"entertainment",
    'regional':'local',
    'sport':'sports',
    'ข่าวกีฬา':'sports',
    'ข่าวอาชญากรรม':'crime',
    'khrongkhaidaasapparod':'ข่าวโครงข่ายตาสับปะรด',
    'around-the-world-news':'foreign',
    'abroad':'foreign',
    'ข่าวต่างประเทศ':'foreign',
    'ข่าวไลฟ์สไตล์':'lifestyle',
    'analysis-today-politics':'politics',
    'politic':'politics',
    'ข่าวเศรษฐกิจ':'economics',
    'economy':'economics',
    'economic':'economics',
    'uncategorized':'others',
    'auto':'car-vehicle',
    'thai-soccer':'thaifootball',
    'ข่าวพระราชสำนัก':'royalnews',
    'it':'tech'
}

In [3]:
def conv_cate(category):
    result = CAT.get(category,-1)
    if result!=-1:
        return result
    else :
        #in case want to update dictionary
        #CAT[category] = category
        return category

In [4]:
df['category'] = df['category'].apply(unquote).apply(conv_cate)

In [5]:
#tags cleaning
#df[df['category']=='1293427']
#df.drop([292],inplace=True)
df['category'].unique()
len(df['category'].unique())

53

In [6]:
df.isnull().sum()
#df.dropna().reset_index(drop=True)

id              0
news_topic      0
news_content    0
date            0
author          0
url             0
category        0
dtype: int64

In [18]:
import numpy as np
category = np.array(df['category'])
tags, freq = np.unique(category,return_counts=True)
freqList = list(zip(tags,freq))
freqList.sort(key=lambda elem:elem[1],reverse=True) #sort by frequency

#get category with lots of data examples
topCats = [category[0] for category in freqList[:9]]
topCats

['local',
 'foreign',
 'politics',
 'sports',
 'entertainment',
 'article',
 'economics',
 'crime',
 'eurofootball']

In [ ]:
df[df.category.isin(topCats)]

In [21]:
X = np.array(df[df.category.isin(topCats)]['news_content'])
y = np.array(df[df.category.isin(topCats)]['category'])

# Cleaning text


In [24]:
re.sub("[^ก-ไ๐-๙‘’“”]", '', 'ฟหกดเ้่าสาสงๆไำพัีนรนบลผปแอิืมทมใฝๅ/t/t_ภถุึคึre.sub("[^ก-ไ๐-๙‘’“”]"ฟหกดเ้่าสาสงๆไำพัีนรนบลผปแอิืมคตจจตช!!!')
#after searching thai ascii

'ฟหกดเาสาสงไำพัีนรนบลผปแอิืมทมใฝภถุึคึกไ๐๙‘’“”ฟหกดเาสาสงไำพัีนรนบลผปแอิืมคตจจตช'

In [23]:
import re
#prepare each sentence
def text_prepare(text):
    #text = re.sub("[/(){}\[\]\|@,;\n\t:\'\-!,'์'‘’“”]", '', text)
    #text = re.sub("[/\xa0]", '', text)
    #text = re.sub("[0-9]", '', text)
    text = re.sub("[^ก-๙‘’“”]", '', text)
    return text

In [25]:
#execute prepare
X = np.array([text_prepare(body) for body in X])


# Prepare train and test set

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [27]:
type(X_train)

numpy.ndarray

In [28]:
len(X_test)

424

## Check tag distribution

In [29]:
#not well distributed
tags, freq = np.unique(y_train,return_counts=True)
list(zip(tags,freq))

[('article', 74),
 ('crime', 60),
 ('economics', 69),
 ('entertainment', 101),
 ('eurofootball', 58),
 ('foreign', 255),
 ('local', 265),
 ('politics', 196),
 ('sports', 192)]

# Tokenize thai words

In [30]:
from pythainlp.tokenize import word_tokenize

In [31]:

#tokenize function
def Thaitokenize(text):
    """
    turn a Thai sentence into Thai words with meaning
    """
    text = ''.join(text.split('"')) ##handled in regex??
    text = word_tokenize(text , engine = 'pyicu')
    return text

In [ ]:
#try mapping function
#Error
#list(map(Thaitokenize,X_train))

In [32]:
Thaitokenize('""แสนสิริ""สร้างUN Global Standards of Conduct for Business\xa0')

['แสน',
 'สิริ',
 'สร้าง',
 'UN',
 ' ',
 'Global',
 ' ',
 'Standards',
 ' ',
 'of',
 ' ',
 'Conduct',
 ' ',
 'for',
 ' ',
 'Business',
 '\xa0']

### just a test for fun###
%timeit word_tokenize('แสนสิริสร้างมาตรฐานใหม่ของการยอมรับความเท่าเทียม' , engine = 'pyicu')

### just a test for fun###
%timeit word_tokenize('แสนสิริสร้างมาตรฐานใหม่ของการยอมรับความเท่าเทียม' , engine = 'newmm')

In [271]:
#words to account in the bag
words_counts = {}

for i in X_train: #each i is a topic sentence
    wordlist = Thaitokenize(i)
    for j in wordlist:
        words_counts[j] = words_counts.get(j,0) + 1
#words_counts = 238

WORDS_TO_INDEX = dict(zip(words_counts.keys(),list(x for x in range(len(words_counts)))))

In [ ]:
len(words_counts)

# Create bag of words

In [ ]:
#create a bag of words for a sentence
def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a sentence
        words_to_index: words to count in the bag 
        e.g.
        words_to_index = {'hi': 0, 'you': 1, 'me': 2, 'are': 3}
        
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'
    """
    result_vector = np.zeros(dict_size)
    
    #tokenize Thai words
    words = Thaitokenize(text)
    
    for i in words:
      if(words_to_index.get(i,-1)!=-1) :
        result_vector[words_to_index[i]]=+1
    return result_vector

In [ ]:
#X_train_my_bag = np.array([my_bag_of_words(text, WORDS_TO_INDEX, len(words_counts)) for text in X_train])

# TFIDF

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [ ]:
#X_train_tfidf = vectorizer.fit_transform(X_train)
#print(vectorizer.get_feature_names())
#print(X.shape)

In [34]:
vectorizer = TfidfVectorizer(tokenizer=Thaitokenize, max_df=0.9, min_df = 5)
#vectorizer = TfidfVectorizer(tokenizer=Thaitokenize, max_df=0.9, min_df = 0)
#max_df=0.8, min_df=5 not much difference
#vectorizer = TfidfVectorizer(tokenizer=Thaitokenize, max_df=0.8, min_df = 10)
X_train_tfidf = vectorizer.fit_transform(X_train)
print(vectorizer.get_feature_names())
print(X_train_tfidf.shape)

['ก', 'กก', 'กค', 'กซ์', 'กฎ', 'กฎหมาย', 'กด', 'กดดัน', 'กต', 'กทม', 'กนก', 'กบ', 'กบฏ', 'กพ', 'กภจว', 'กม', 'กมล', 'กย', 'กร', 'กรกฎาคม', 'กรง', 'กรณี', 'กรม', 'กรมวิทยาศาสตร์การแพทย์', 'กรมอุตุนิยมวิทยา', 'กรรม', 'กรรมการ', 'กรรมการผู้จัดการ', 'กรรมาธิการ', 'กรวม', 'กรอง', 'กรอบ', 'กระ', 'กระจก', 'กระจาย', 'กระชับ', 'กระชาก', 'กระดูก', 'กระตุ้น', 'กระทบ', 'กระทรวง', 'กระทรวงกลาโหม', 'กระทรวงการคลัง', 'กระทรวงการต่างประเทศ', 'กระทรวงคมนาคม', 'กระทรวงพาณิชย์', 'กระทรวงมหาดไทย', 'กระทรวงยุติธรรม', 'กระทรวงศึกษาธิการ', 'กระทรวงสาธารณสุข', 'กระทรวงอุตสาหกรรม', 'กระทรวงเกษตรฯ', 'กระทรวงเกษตรและสหกรณ์', 'กระทั่ง', 'กระทำ', 'กระทำความผิด', 'กระทิง', 'กระบวน', 'กระบวนการ', 'กระบะ', 'กระบี่', 'กระสุน', 'กระสุนปืน', 'กระเด็น', 'กระเป๋า', 'กระแทก', 'กระแส', 'กระแสข่าว', 'กระโดด', 'กรัม', 'กราด', 'กราบ', 'กราย', 'กรีด', 'กรีน', 'กรี๊ด', 'กรุง', 'กรุงเทพ', 'กรุงเทพมหานคร', 'กรุงเทพฯ', 'กรุณา', 'กรุ๊ป', 'กฤษ', 'กลยุทธ์', 'กลับ', 'กลับคืน', 'กลับบ้าน', 'กลับมา', 'กลัว', 'กลั่น', 'กลาง', 'กลางคืน', '

In [ ]:
#[word for word in WORDS_TO_INDEX]
#type(WORDS_TO_INDEX.keys())
#X_train_tfidf

##ARRAY <-> Sparse matrix much faster
#X_train_tfidf.toarray() 
#sparse matrix to array
#from scipy import sparse
#sparse.csr_matrix(X_train_tfidf.toarray()) 
#array to sparse matrix

In [ ]:
#for illustration & checking
featurelist = vectorizer.get_feature_names()
TFIDF = pd.DataFrame(X_train_tfidf.toarray(),columns = featurelist)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
TFIDF


In [35]:
X_train_tfidf

<1270x5152 sparse matrix of type '<class 'numpy.float64'>'
	with 165188 stored elements in Compressed Sparse Row format>

# turn y values to 1s,0s

In [ ]:
# from sklearn.preprocessing import MultiLabelBinarizer
# this is for list of tags

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
#lb.fit_transform([1, 2, 6, 4, 2]) 
#fit & transform in one step
#fit = figure out different classes and turn into necessary binary formats
#transform = transform given data into binary formats

In [ ]:
y_train_lb = lb.fit_transform(y_train)


In [ ]:
lb.classes_.size

In [ ]:
#for illustration & checking
Y_LB = pd.DataFrame(y_train_lb,columns = lb.classes_)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
Y_LB.iloc[10:20]
#y_train.iloc[10:20]

In [ ]:
y_train_lb.shape

# Train model to predict tags

In [36]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
#example
X = np.array([
     [10, 10],
     [8, 10],
     [-5, 5.5],
     [-5.4, 5.5],
     [-20, -20],
     [-15, -20]])
y = np.array([0, 0, 1, 1, 2, 2])
clf = OneVsRestClassifier(SVC()).fit(X, y)
clf.predict([[-19, -20], [9, 9], [-5, 5]])

array([2, 0, 1])

In [37]:
clf = OneVsRestClassifier(SVC()).fit(X_train_tfidf, y_train)

In [ ]:
y_train

In [38]:
#check the predictions, models applied correctly? Yes
y = clf.predict(X_train_tfidf)
print(np.concatenate((y.reshape(len(y),1), y_train.reshape(len(y_train),1)),1))

[['politics' 'politics']
 ['politics' 'politics']
 ['local' 'local']
 ...
 ['article' 'article']
 ['foreign' 'foreign']
 ['entertainment' 'entertainment']]


In [39]:
X_test = np.array([text_prepare(title) for title in X_test])
#X_test_my_bag = np.array([my_bag_of_words(text, WORDS_TO_INDEX, len(words_counts)) for text in X_test])
X_test_tfidf = vectorizer.transform(X_test)
y_pred = clf.predict(X_test_tfidf)
y_test = np.array(y_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
#print(np.concatenate(y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1),1))

[['politics' 'economics']
 ['entertainment' 'entertainment']
 ['local' 'article']
 ['sports' 'sports']
 ['foreign' 'foreign']
 ['crime' 'crime']
 ['local' 'local']
 ['entertainment' 'entertainment']
 ['politics' 'politics']
 ['foreign' 'foreign']
 ['foreign' 'foreign']
 ['entertainment' 'entertainment']
 ['local' 'local']
 ['politics' 'politics']
 ['foreign' 'foreign']
 ['foreign' 'foreign']
 ['foreign' 'foreign']
 ['local' 'crime']
 ['local' 'local']
 ['local' 'crime']
 ['crime' 'local']
 ['politics' 'politics']
 ['local' 'local']
 ['local' 'crime']
 ['crime' 'crime']
 ['foreign' 'foreign']
 ['local' 'local']
 ['foreign' 'foreign']
 ['local' 'local']
 ['foreign' 'foreign']
 ['local' 'local']
 ['local' 'local']
 ['crime' 'crime']
 ['foreign' 'foreign']
 ['sports' 'sports']
 ['sports' 'sports']
 ['article' 'sports']
 ['politics' 'economics']
 ['foreign' 'sports']
 ['eurofootball' 'eurofootball']
 ['local' 'local']
 ['economics' 'local']
 ['politics' 'politics']
 ['economics' 'economics'

In [40]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 3  0  0  4  0  2  9  6  0]
 [ 0  7  0  0  0  0 13  0  0]
 [ 0  0 14  0  0  4  5  4  0]
 [ 2  0  0 16  0  2  7  0  1]
 [ 0  0  0  0 10  0  0  0 10]
 [ 0  0  0  0  0 80  3  0  0]
 [ 0  1  3  1  0  3 74  9  1]
 [ 0  0  1  0  0  3  2 58  1]
 [ 1  0  0  0  0  2  0  0 62]]


0.7641509433962265

In [41]:
test = text_prepare('“อนุทิน” ขอ ผู้ร้อง เปิดชื่อ อักษรย่อ \"พ\" คนหักหัวคิวโรงแรม 40%')
#test = np.array([my_bag_of_words(test, WORDS_TO_INDEX, len(words_counts))])
test = vectorizer.transform([test])
clf.predict(test)

array(['politics'], dtype='<U13')